# Creating a live dashboard using Twitch API

+ Twitch API let's you access live game, stream, and user-level data. In this project, we are trying to analyze the viewers for top streamers and how this variation changes through the factors like game, language, and country.
+ Initial ideas are to create a live visualization that displays game level data, and top streamers within that.

- Vardayini, Maddhujeet, and Kaivalya

## Learning to access Twitch API

### Importing libraries

In [6]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import threading

### Acquiring access code from twitch using Client ID & Client Secret

In [2]:
#Kaivalya's Client ID: --***---
#Kaivalya's Client Secret: ---***---

client_id = ''
client_secret = ''

access_code = requests.post('https://id.twitch.tv/oauth2/token?client_id='+str(client_id)+'&client_secret='+str(client_secret)+'&grant_type=client_credentials')
           

In [ ]:
token = json.loads(access_code.text)
access_token = token['access_token']

In [5]:
access_code

<Response [200]>

In [14]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
}
response = requests.get('https://api.twitch.tv/helix/streams?first=100', headers=headers)

In [15]:
#print(response.text)
response_json = json.loads(response.text)

In [ ]:
data = response_json['data']

df = pd.DataFrame.from_dict(json_normalize(data), orient='columns')

In [17]:
df.head()

,id,user_id,user_name,game_id,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids
0,36363765120,36029255,Riot Games,21779,live,2019 All-Star Event: LCK vs. LPL,60512,2019-12-06T22:56:18Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039]
1,36363991600,190835892,LCK_Korea,21779,live,2019 LoL 올스타전 2일차,32804,2019-12-06T23:30:04Z,ko,https://static-cdn.jtvnw.net/previews-ttv/live...,[ab2975e3-b9ca-4b1a-a93e-fb61a5d5c3a4]
2,36362725360,71092938,xQcOW,458641,live,OMG! IS IT TIME! ALREDY! AHHHH!,20544,2019-12-06T19:24:07Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[6ea6bca4-4712-4ab9-a906-e3336a9d8039, 1eba3cf..."
3,36363164752,29795919,nl_Kripp,138585,live,KRIPP BATTLEGROUNDS ALL DAY | Rafaam OP! youtu...,15075,2019-12-06T21:07:32Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[6ea6bca4-4712-4ab9-a906-e3336a9d8039, 7f61316..."
4,36364530992,163836275,Lord_Kebun,32982,live,Mr.Chang | NoPixel RP | Twitter @LordKebun,13856,2019-12-07T00:42:41Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039]


### Creating a CSV for Stream data

In [9]:
export_csv = df.to_csv (r'/Users/kaivalya/Twitch/first.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


# Saved to local directory:/Users/kaivalya/Twitch/ as first.csv

## Accessing Game data 

### Making a list of all games in the top 100 streams

In [10]:
game_ids = list(df['game_id'])
game_ids = '&id='.join(game_ids)

In [11]:
game_ids

'417969&id=21779&id=509658&id=0&id=33214&id=29595&id=503446&id=512710&id=489635&id=138585&id=509658&id=21779&id=32982&id=21779&id=509658&id=138585&id=509658&id=509658&id=21779&id=512710&id=32399&id=29595&id=138585&id=509658&id=21779&id=29595&id=263490&id=513143&id=489635&id=21779&id=509658&id=33214&id=33214&id=488552&id=&id=511399&id=18122&id=65632&id=21779&id=21779&id=509658&id=33214&id=33214&id=512710&id=65632&id=489635&id=512710&id=460630&id=512804&id=511224&id=509658&id=511224&id=509658&id=512526&id=65632&id=65632&id=29595&id=488190&id=509658&id=488190&id=29595&id=509658&id=138585&id=509658&id=509658&id=32399&id=511399&id=509658&id=512710&id=138585&id=509672&id=138585&id=21779&id=513143&id=497451&id=491487&id=29595&id=498566&id=509658&id=21779&id=29307&id=417752&id=18122&id=21779&id=509658&id=21779&id=488190&id=21779&id=21779&id=29595&id=512988&id=512804&id=497451&id=512710&id=26936&id=21779&id=21779&id=32982&id=493959&id=493057'

### Passing the list of 100 games to get the game's name from its ID

In [ ]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
}
response_games = requests.get('https://api.twitch.tv/helix/games?id='+game_ids, headers=headers)

response_games_json = json.loads(response_games.text)

### Converting game data json into a dataframe

In [13]:
data_games = response_games_json['data']

games_df = pd.DataFrame.from_dict(json_normalize(data_games), orient='columns')
games_df.head()

,id,name,box_art_url
0,513143,Teamfight Tactics,https://static-cdn.jtvnw.net/ttv-boxart/Teamfi...
1,488552,Overwatch,https://static-cdn.jtvnw.net/ttv-boxart/Overwa...
2,29595,Dota 2,https://static-cdn.jtvnw.net/ttv-boxart/Dota%2...
3,460630,Tom Clancy's Rainbow Six: Siege,https://static-cdn.jtvnw.net/ttv-boxart/./Tom%...
4,512710,Call of Duty: Modern Warfare,https://static-cdn.jtvnw.net/ttv-boxart/./Call...


### Converting the df into a csv to explore in Tableau

In [14]:
export_games_csv = games_df.to_csv (r'/Users/kaivalya/Twitch/first_games.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

# Saved in local directory: /Users/kaivalya/Twitch/ as first_games.csv

### repeatable code

In [2]:

def twitch():
    threading.Timer(60.0, twitch).start()
    
    headers = {
        'Authorization' : 'Bearer '+str(access_token),
    }
    response = requests.get('https://api.twitch.tv/helix/streams?first=100', headers=headers)

    response_json = json.loads(response.text)
    data = response_json['data']
    df = pd.DataFrame.from_dict(json_normalize(data), orient='columns')
    export_csv = df.to_csv (r'/Users/kaivalya/Twitch/first.csv', index = None, header=True)
    
    game_ids = list(df['game_id'])
    game_ids = '&id='.join(game_ids)

    headers = {
        'Authorization' : 'Bearer '+str(access_token),
    }
    response_games = requests.get('https://api.twitch.tv/helix/games?id='+game_ids, headers=headers)

    #print(response_games.text)
    response_games_json = json.loads(response_games.text)
    data_games = response_games_json['data']
    games_df = pd.DataFrame.from_dict(json_normalize(data_games), orient='columns')
    export_games_csv = games_df.to_csv (r'/Users/kaivalya/Twitch/first_games.csv', index = None, header=True)

twitch()

## Exploring other APIs

### Getting Streamers information

In [18]:
user_ids = list(df['user_id'])
user_ids = '&id='.join(user_ids)
user_ids

'36029255&id=190835892&id=71092938&id=29795919&id=163836275&id=26490481&id=121059319&id=28462004&id=41783889&id=36511475&id=79854877&id=120244187&id=93053674&id=104833324&id=22346597&id=192678094&id=63602976&id=54706574&id=28036688&id=32140000&id=52614128&id=69906737&id=31239503&id=26707340&id=51914127&id=39885827&id=11249217&id=38251312&id=20993498&id=29289159&id=7010591&id=20786541&id=93049594&id=30672329&id=29899360&id=38594688&id=207813352&id=23523971&id=24811779&id=28379957&id=117379932&id=71852533&id=18074328&id=61433001&id=63164470&id=9679595&id=40336240&id=1518077&id=28219022&id=47702337&id=63880003&id=48281577&id=137640527&id=34932688&id=134826318&id=20650414&id=133220545&id=121063510&id=181077473&id=155601320&id=50237189&id=36620767&id=149188013&id=464134384&id=84316241&id=36513760&id=39724467&id=15554591&id=23852822&id=79615025&id=26469355&id=61504845&id=474745252&id=38881685&id=28703999&id=40754777&id=96858382&id=94875296&id=198506129&id=68292428&id=205301469&id=15027034&id

In [19]:
headers = {
    'Authorization': 'Bearer '+str(access_token),
}
response_user_ids = requests.get('https://api.twitch.tv/helix/users?id='+user_ids, headers=headers)

In [20]:
response_user_json = json.loads(response_user_ids.text)
data_user = response_user_json['data']
users_df = pd.DataFrame.from_dict(json_normalize(data_user), orient='columns')

users_df.head()

,id,login,display_name,type,broadcaster_type,description,profile_image_url,offline_image_url,view_count
0,36029255,riotgames,Riot Games,,partner,"Welcome to the Riot Games channel, home of the...",https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,1214850084
1,190835892,lck_korea,LCK_Korea,,partner,,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,118223163
2,71092938,xqcow,xQcOW,,partner,Overwatch Professional tank player and full ti...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,103410236
3,29795919,nl_kripp,nl_Kripp,,partner,Gamer YouTuber Streamer Joker,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,230645297
4,163836275,lord_kebun,Lord_Kebun,,partner,Always keep it entertaining,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,19013922


In [22]:
users_df['broadcaster_type'].value_counts()

partner      92
affiliate     4
              2
Name: broadcaster_type, dtype: int64

## pagination & game data

In [7]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
}
games_response = requests.get('https://api.twitch.tv/helix/games/top?first=100', headers=headers)

In [8]:
#print(response.text)
games_response_json = json.loads(games_response.text)

topgames_data = games_response_json['data']

topgames_df = pd.DataFrame.from_dict(json_normalize(topgames_data), orient='columns')
topgames_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
id             99 non-null object
name           99 non-null object
box_art_url    99 non-null object
dtypes: object(3)
memory usage: 2.4+ KB


In [9]:
topgames_df.head()

,id,name,box_art_url
0,33214,Fortnite,https://static-cdn.jtvnw.net/ttv-boxart/Fortni...
1,21779,League of Legends,https://static-cdn.jtvnw.net/ttv-boxart/League...
2,509658,Just Chatting,https://static-cdn.jtvnw.net/ttv-boxart/Just%2...
3,512070,Jump King,https://static-cdn.jtvnw.net/ttv-boxart/Jump%2...
4,32982,Grand Theft Auto V,https://static-cdn.jtvnw.net/ttv-boxart/Grand%...


In [124]:
topgames_data = games_response_json['pagination']
pagination= topgames_data['cursor']
print(pagination)
print(topgames_data)
str(pagination)

eyJzIjoxMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ==
{'cursor': 'eyJzIjoxMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ=='}


'eyJzIjoxMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ=='

In [129]:
topgames_data = games_response_json['pagination']
pagination= topgames_data['cursor']
print(pagination)
print(topgames_data)
str(pagination)

headers = {
    'Authorization' : 'Bearer '+str(access_token),
}
games_response = requests.get('https://api.twitch.tv/helix/games/top?first=100&after='+str(pagination), headers=headers)

eyJzIjoyMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ==
{'cursor': 'eyJzIjoyMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ=='}


In [130]:
#print(response.text)
games_response_json = json.loads(games_response.text)

topgames_data = games_response_json['data']
topgames_df_temp = pd.DataFrame.from_dict(json_normalize(topgames_data), orient='columns')
topgames_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
id             100 non-null object
name           100 non-null object
box_art_url    100 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [131]:
print(games_response)

<Response [200]>


In [132]:
#topgames_df.append(topgames_df_temp, ignore_index=True, sort=False)
frames = [topgames_df, topgames_df_temp]
topgames_df = pd.concat(frames, ignore_index=True)
topgames_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 3 columns):
id             299 non-null object
name           299 non-null object
box_art_url    299 non-null object
dtypes: object(3)
memory usage: 7.1+ KB


In [149]:
export_topgames_csv = topgames_df.to_csv (r'/Users/kaivalya/Twitch/topgames.csv', index = None, header=True)

### getting stream data for the games

In [10]:
topstreamsforgame_ids = topgames_df['id']
topstreamsforgame_ids[0]

'33214'

In [11]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
}
topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][0])+'&first=100', headers=headers)


In [12]:
print(topstreamsforgame_response)

<Response [200]>


In [ ]:
topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)

topstreamsforgame_data = topstreamsforgame_response_json['data']

topstreamsforgame_df = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')
topstreamsforgame_df.head()

### Automating the code pull

Automating the pulling of top streamer data for top games. I put in a sleep timer for 30 seconds, so this snippet takes about 20 minutes to run depending on the recurrence of for loops

In [14]:
# REALLY LONG CODE
#Accessing API
headers = {
    'Authorization' : 'Bearer '+str(access_token),
}
topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][0])+'&first=25', headers=headers)

# Converting to JSON
topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)

# Getting data from the JSON
topstreamsforgame_data = topstreamsforgame_response_json['data']

# Converting into a DataFrame
topstreamsforgame_df = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')

# Automated flow
for i in range(1,19) :
    
    headers = {
        'Authorization' : 'Bearer '+str(access_token),
    }
    topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][i])+'&first=25', headers=headers)
    
    topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)
    topstreamsforgame_data = topstreamsforgame_response_json['data']
    topstreamsforgame_df_temp = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')
    
    frames = [topstreamsforgame_df, topstreamsforgame_df_temp]
    topstreamsforgame_df = pd.concat(frames, ignore_index=True)
    
    #time.sleep(30)

topstreamsforgame_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 11 columns):
id               475 non-null object
user_id          475 non-null object
user_name        475 non-null object
game_id          475 non-null object
type             475 non-null object
title            475 non-null object
viewer_count     475 non-null int64
started_at       475 non-null object
language         475 non-null object
thumbnail_url    475 non-null object
tag_ids          475 non-null object
dtypes: int64(1), object(10)
memory usage: 40.9+ KB


In [16]:
topstreamsforgame_df.head()

,id,user_id,user_name,game_id,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids
470,36437716432,58999722,snowlit,497451,live,❄️ Do I still have any shiny hunting luck? (1s...,126,2019-12-17T20:58:23Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[6ea6bca4-4712-4ab9-a906-e3336a9d8039, d24d840..."
471,36438868592,90699866,pokeguy,497451,live,Pokemon Shield Speedruns,104,2019-12-17T23:58:48Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039]
472,36439410656,108392761,JackAM,497451,live,Jack Plays Pokemon,101,2019-12-18T01:03:02Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[6ea6bca4-4712-4ab9-a906-e3336a9d8039, 77017b7..."
473,36439924208,127247843,Choctopus,497451,live,Viewer Raids N Battles,100,2019-12-18T02:01:13Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[6ea6bca4-4712-4ab9-a906-e3336a9d8039, ac763b1..."
474,36438948144,124889479,Ozzy,497451,live,✨GOT SHINY NICKIT✨Hunting Impidimp,95,2019-12-18T00:07:49Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[8ba227ca-073c-46a7-b3cc-193e52c5ab4d, 27937ce..."


In [148]:
export_topstreamsforgame_csv = topstreamsforgame_df.to_csv (r'/Users/kaivalya/Twitch/topstreamsforgame.csv', index = None, header=True)

## Streamers follows

This API call gives us information about a streamers followers and the people they followed. Tried to explore how this information could be used to describe streamer or viewer behavior.

In [37]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
}
followers_response = requests.get('https://api.twitch.tv/helix/users/follows?from_id=23161357&first=100', headers=headers)


In [ ]:
followers_response_json = json.loads(followers_response.text)

followers_data = followers_response_json['data']
followers_df = pd.DataFrame.from_dict(json_normalize(followers_data), orient='columns')
print(followers_df)

In [39]:
followers_data = followers_response_json['pagination']
followers_pagination= followers_data['cursor']
print(followers_pagination)
print(followers_data)
str(followers_pagination)

eyJiIjpudWxsLCJhIjp7IkN1cnNvciI6IjEzNjQzOTk3NDY0ODc1NDkwMDAifX0
{'cursor': 'eyJiIjpudWxsLCJhIjp7IkN1cnNvciI6IjEzNjQzOTk3NDY0ODc1NDkwMDAifX0'}


'eyJiIjpudWxsLCJhIjp7IkN1cnNvciI6IjEzNjQzOTk3NDY0ODc1NDkwMDAifX0'

In [40]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
}
followers_response = requests.get('https://api.twitch.tv/helix/users/follows?from_id=23161357&first=100&after'+str(followers_pagination), headers=headers)

In [41]:
followers_response_json = json.loads(followers_response.text)

followers_data = followers_response_json['data']
followers_df_temp = pd.DataFrame.from_dict(json_normalize(followers_data), orient='columns')
print(followers_df_temp)

     from_id from_name      to_id           to_name           followed_at
0   23161357     LIRIK  117379932        KingGeorge  2019-10-02T04:06:19Z
1   23161357     LIRIK  110690086              Myth  2019-08-28T02:53:17Z
2   23161357     LIRIK  450512030           Porsche  2019-08-27T22:19:36Z
3   23161357     LIRIK  254151744     adorablestars  2019-08-10T16:06:55Z
4   23161357     LIRIK   49946840  CyberPowerGaming  2019-08-09T20:59:58Z
5   23161357     LIRIK   71190292     Trainwreckstv  2019-07-27T02:58:30Z
6   23161357     LIRIK   43830727     KingGothalion  2019-07-19T16:14:45Z
7   23161357     LIRIK   21841789            Nmplol  2019-05-15T15:38:54Z
8   23161357     LIRIK   40972890     AdmiralBahroo  2019-05-01T17:03:31Z
9   23161357     LIRIK   36841405       MikeTheBard  2019-04-10T22:34:57Z
10  23161357     LIRIK  163836275        Lord_Kebun  2019-04-10T02:13:36Z
11  23161357     LIRIK   12943173         pokelawls  2019-04-06T19:52:08Z
12  23161357     LIRIK  136765278     